In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.datasets import mnist

Using TensorFlow backend.
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\admin\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# Loading our datasets -- containing images of handwritten digits 0f 0 - 9
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 19s 2us/step


In [3]:
# Input image dimensions
img_width, img_height = 28, 28

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_width, img_height)
    x_test = x_test.reshape(x_test.shape[0], 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else:
    x_train = x_train.reshape(x_train.shape[0], img_width, img_height, 1)
    x_test = x_test.reshape(x_test.shape[0], img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
# Normalizing our dataets
x_train = x_train / 255
x_test = x_test / 255

train_y = to_categorical(y_train)
test_y = to_categorical(y_test)

print("x_train shape is: ", x_train.shape)
print("x_test shape is: ", x_test.shape)
print("train_y shape is: ", train_y.shape)
print("test_y shape is: ", test_y.shape)

x_train shape is:  (60000, 28, 28, 1)
x_test shape is:  (10000, 28, 28, 1)
train_y shape is:  (60000, 10)
test_y shape is:  (10000, 10)


In [7]:
# Building our model
model = Sequential()
model.add(Convolution2D(filters = 32, kernel_size = (5, 5), input_shape = input_shape, activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Convolution2D(filters = 64, kernel_size = (5, 5), input_shape = input_shape, activation = "relu"))
model.add(MaxPooling2D(pool_size = (3, 3)))
model.add(Flatten()) # reducing the dimension into 1D 
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))  # 10---> num of clases,i.e, 0-9

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [9]:
# Compiling our model
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
model.fit(x_train, train_y, validation_data = (x_test, test_y), epochs = 12, batch_size = 64)
scores = model.evaluate(x_test, test_y)
print("CNN Error : %.2f%%" % (100- scores[1] * 100))
model.save("digit_classifier.mnist")


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 93s 2ms/step - loss: 0.2139 - accuracy: 0.9355 - val_loss: 0.0638 - val_accuracy: 0.9800
Epoch 2/12
60000/60000 [==============================] - 98s 2ms/step - loss: 0.0661 - accuracy: 0.9800 - val_loss: 0.0425 - val_accuracy: 0.9868
Epoch 3/12
60000/60000 [==============================] - 97s 2ms/step - loss: 0.0469 - accuracy: 0.9855 - val_loss: 0.0334 - val_accuracy: 0.9883
Epoch 4/12
60000/60000 [==============================] - 88s 1ms/step - loss: 0.0360 - accuracy: 0.9886 - val_loss: 0.0314 - val_accuracy: 0.9907
Epoch 5/12
60000/60000 [==============================] - 91s 2ms/step - loss: 0.0297 - accuracy: 0.9904 - val_loss: 0.0275 - val_accuracy: 0.9906
Epoch 6/12
59968/60000 [============================>.] - ETA: 0s - loss: 0.0256 - accuracy: 0.9922